In [1]:
import os

import pickle

import numpy as np
import torch

import open3d as o3d

import matplotlib.pyplot as plt
import matplotlib.cm as cm
cmap = cm.get_cmap('jet')

from tqdm.notebook import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/tmp/ipykernel_208500/1176535136.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('jet')


In [2]:
import ipywidgets as widgets
from IPython.display import display

In [3]:
MATTERPORT_DIR = "/media/rsl_admin/T7/matterport/data/v1/scans"

# SEGMENTATIONS_PATH = "/home/rsl_admin/openscene/comparison_outputs/segmentations/object-openseg-matterport-test"
SEGMENTATIONS_PATH = "/home/rsl_admin/openscene/comparison_outputs/segmentations/region-openseg-matterport-test"

# SEGMENTATIONS_PATH = "/home/rsl_admin/openscene/comparison_outputs/segmentations/object-openseg-matterport-val"
# SEGMENTATIONS_PATH = "/home/rsl_admin/openscene/comparison_outputs/segmentations/region-openseg-matterport-val"

### Get list of scans within this directory

In [4]:
scan_names = set()
for fname in os.listdir(SEGMENTATIONS_PATH):
    name = fname.split("_")[0]
    if name not in scan_names:
        scan_names.add(name)
        
print(scan_names)

{'Vt2qJdWjCF2', 'gYvKGZ5eRqb', '5ZKStnWn8Zo', 'wc2JMjhGNzB', 'ARNzJeq3xxb', 'fzynW3qQPVF', 'YVUC4YcDtcY', 'gxdoqLR6rwA', 'yqstnuAEVhm', 'pa4otMbVnkk', 'YFuZgdQ5vWj', 'rqfALeAoiTq', '2t7WUuJeko7', 'UwV83HsGsw3', 'RPmz2sHmrrY', 'WYY7iVyf5p8', 'jtcxE69GiFV', 'q9vSo1VnCiC'}


In [5]:
# scan_name = np.random.choice(list(scan_names))

scan_name = "2t7WUuJeko7"

In [6]:
points = []
colors = []
preds = []
label_to_ind = None

for fname in os.listdir(SEGMENTATIONS_PATH):
    if scan_name not in fname:
        continue
        
    with open(os.path.join(SEGMENTATIONS_PATH, fname), 'rb') as f:
        data = pickle.load(f)
        
    points.append(data["points"])
    colors.append(data["colors"])
    preds.append(data["preds"])
    
    # This assumes that label_to_ind is the same for all regions preds generated
    if label_to_ind == None:
        label_to_ind = data["label_to_ind"]
        
        
points = np.concatenate(points, axis=0)
colors = np.concatenate(colors, axis=0)
preds = np.concatenate(preds, axis=0)

In [7]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

In [8]:
scan_mesh = o3d.io.read_triangle_mesh(
    os.path.join(MATTERPORT_DIR, f"{scan_name}/{scan_name}/house_segmentations/{scan_name}.ply")
)

##
Visualize the scan mesh and aggregated points from all section

In [9]:
o3d.visualization.draw_geometries([scan_mesh, pcd])

##
Visualize for a selected label the segmentations

In [10]:
labels = label_to_ind.keys()

print(labels)

dict_keys(['bathroom', 'bedroom', 'closet', 'dining room', 'garage', 'hallway', 'library', 'laundryroom/mudroom', 'kitchen', 'living room', 'meetingroom/conferenceroom', 'office', 'porch/terrace/deck/driveway', 'rec/game', 'stairs', 'utilityroom/toolroom', 'tv', 'workout/gym/exercise', 'balcony', 'bar', 'classroom', 'spa/sauna', 'entryway/foyer/lobby', 'outdoor', 'dining booth', 'other room'])


In [11]:
label_dropdown = widgets.Dropdown(
    options=['all'] + sorted(labels),
    disabled=False,
)

display(label_dropdown)

Dropdown(options=('all', 'balcony', 'bar', 'bathroom', 'bedroom', 'classroom', 'closet', 'dining booth', 'dini…

In [12]:
label_inds = np.where(preds == label_to_ind[label_dropdown.value])[0]

o3d.visualization.draw_geometries(
    [
        scan_mesh, 
        pcd.select_by_index(label_inds).paint_uniform_color((0,1,0))
    ]
)

KeyError: 'all'

##
## See what the clusters look like

In [13]:
BLACKLISTED_LABELS_OBJECTS = (
    "misc", "objects", "void", "unlabeled",
    "wall", "floor", "ceiling",
)

BLACKLISTED_LABELS_REGIONS = (
    "other room", "junk", "no label",

    # no appropriate tag
    "dining booth",
    "entryway/foyer/lobby",
    "outdoor",
)

BLACKLISTED_LABELS = set(BLACKLISTED_LABELS_OBJECTS).union(BLACKLISTED_LABELS_REGIONS)

print(BLACKLISTED_LABELS)

{'void', 'objects', 'unlabeled', 'ceiling', 'dining booth', 'no label', 'entryway/foyer/lobby', 'other room', 'wall', 'junk', 'floor', 'misc', 'outdoor'}


In [34]:
DBSCAN_PARAMS = {
#     "eps": 0.1,   # what's used in OVIR paper
#     "eps": 0.2,
    
    "eps": 0.5,     # for regions
    
    "min_points": 1,
}

In [35]:
label_proposals = {}

for label in tqdm(labels):
    if label in BLACKLISTED_LABELS:
        continue
    
    
    label_points_pcd = o3d.geometry.PointCloud()
    label_points_pcd.points = o3d.utility.Vector3dVector(
        points[preds == label_to_ind[label]]
    )
    
    
    cluster_inds = label_points_pcd.cluster_dbscan(**DBSCAN_PARAMS)
    
    
    label_proposals[label] = {
        "points": [],
        "boxes": [],
    }
    
    for i in np.unique(cluster_inds):
        if i == -1:
            continue
        
        if np.sum(cluster_inds == i) < 50:
            continue
        
        cluster_points = label_points_pcd.select_by_index(
            np.where(cluster_inds == i)[0]
        )
        
        cluster_box = cluster_points.get_axis_aligned_bounding_box()
        
        
        label_proposals[label]["points"].append(cluster_points)
        label_proposals[label]["boxes"].append(cluster_box)
        

  0%|          | 0/26 [00:00<?, ?it/s]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


In [36]:
options = [
    f"{len(lp['boxes'])}-{l}" for l, lp in label_proposals.items()
]

label_cluster_dropdown = widgets.Dropdown(
    options=options,
    disabled=False,
)

display(label_cluster_dropdown)

Dropdown(options=('1-bathroom', '4-bedroom', '13-closet', '11-dining room', '0-garage', '0-hallway', '0-librar…

In [38]:
selected_label = label_cluster_dropdown.value.split("-")[-1]

for pcd, box in zip(
    label_proposals[selected_label]["points"],
    label_proposals[selected_label]["boxes"],
):
    color = np.random.uniform(size=3)
    pcd.paint_uniform_color(color)
    box.color = color
    
o3d.visualization.draw_geometries(
    [scan_mesh] + \
    label_proposals[selected_label]["points"] + \
    label_proposals[selected_label]["boxes"]
)